In [ ]:
from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [ ]:
llm = GooglePalm(temperature=0.1)

In [ ]:
poem = llm("Write a 4 line poem of my love for samosa")
poem

In [ ]:
essay = llm("write email requesting refund for electronic item")
print(essay)

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./faqs.csv", source_column="prompt")
data = loader.load()
data

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import GooglePalmEmbeddings

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is your refund policy?")

In [ ]:
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(documents=data, embeddings=instructor_embeddings)
retriever = FAISS.as_retriever(score_threshold=0.7)

In [ ]:
rdocs = retriever.get_relevant_documents("How Long Is This Course?")
rdocs

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    input_key="query",
                                    return_source_documents=True,
                                    chain_type_kwargs=chain_type_kwargs)

In [ ]:
chain('Do you provide job assistance and also do you provide job gurantee?')

In [ ]:
chain("Do you guys provide internship and also do you offer EMI payments?")

In [ ]:
chain("do you have javascript course?")

In [ ]:
chain("Do you have plans to launch blockchain course in future?")

In [ ]:
chain("should I learn power bi or tableau?")

In [ ]:
chain("I've a MAC computer. Can I use powerbi on it?")

In [ ]:
chain("I don't see power pivot. how can I enable it?")

In [ ]:
chain("What is the price of your machine learning course?")